# tf.dataを使ってpandasのDataFrameをロードする

このチュートリアルでは、pandasのDataFrameをロードして、`tf.data.Dataset`にデータを読み込む。

このチュートリアルでは、クリーブランドクリニック財団(the Cleveland Clinic Foundation for Heart Disease)から提供された、小さなデータセットを使用する。
このデータセット(CSV)には、数百行のデータが含まれている。行は各患者を、列は様々な属性を表している。

このデータを使って、患者が心臓病を罹患しているかを判別予測することができる。
なお、この問題は、二値分類問題である。

## pandasを使ってデータを読み込む

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf

In [2]:
csv_file = tf.keras.utils.get_file("heart.csv", "https://storage.googleapis.com/applied-dl/heart.csv")

In [3]:
df = pd.read_csv(csv_file)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [4]:
df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal         object
target        int64
dtype: object

dataframe内で唯一の`object`型である`thal`列を離散値に変換する。

In [5]:
# print(df["thal"])
# pd.Categorical(df["thal"])
df["thal"] = pd.Categorical(df["thal"])
df["thal"] = df.thal.cat.codes

In [6]:
df.head

<bound method NDFrame.head of      age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   1       145   233    1        2      150      0      2.3   
1     67    1   4       160   286    0        2      108      1      1.5   
2     67    1   4       120   229    0        2      129      1      2.6   
3     37    1   3       130   250    0        0      187      0      3.5   
4     41    0   2       130   204    0        2      172      0      1.4   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   52    1   1       118   186    0        2      190      0      0.0   
299   43    0   4       132   341    1        2      136      1      3.0   
300   65    1   4       135   254    0        2      127      0      2.8   
301   48    1   4       130   256    1        2      150      1      0.0   
302   63    0   4       150   407    0        2      154      0      4.0   

     slope  ca  thal  target  
0        3   0     2      

## tf.data.Datasetを使ってデータをロードする

`tf.data.Dataset.from_tensor_slices`メソッドを使って、pandasのdataframeから値を読み込む。

`tf.data.Dataset`を使う利点は、シンプルに使えて、かつ、大変効率的なデータパイプラインを構築できること。詳細は以下のloading data guideを参照。

[loading data guide](https://www.tensorflow.org/guide/data?hl=ja)


In [7]:
target = df.pop("target")
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

for feat, targ in dataset.take(5):
    print("Features: {}, Target: {}".format(feat, targ))

Features: [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   2. ], Target: 0
Features: [ 67.    1.    4.  160.  286.    0.    2.  108.    1.    1.5   2.    3.
   3. ], Target: 1
Features: [ 67.    1.    4.  120.  229.    0.    2.  129.    1.    2.6   2.    2.
   4. ], Target: 0
Features: [ 37.    1.    3.  130.  250.    0.    0.  187.    0.    3.5   3.    0.
   3. ], Target: 0
Features: [ 41.    0.    2.  130.  204.    0.    2.  172.    0.    1.4   1.    0.
   3. ], Target: 0


`pd.Series`は`__array__`プロトコルを実装しているため、`np.array`や`tf.Tensor`を使うところでは、だいたいどこでも使うことができる。

In [8]:
tf.constant(df["thal"])

<tf.Tensor: shape=(303,), dtype=int32, numpy=
array([2, 3, 4, 3, 3, 3, 3, 3, 4, 4, 2, 3, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3,
       3, 4, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 4, 2, 4, 3, 4, 3, 4, 4,
       2, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 4,
       4, 2, 3, 3, 4, 3, 4, 3, 3, 4, 4, 3, 3, 4, 4, 3, 3, 3, 3, 4, 4, 4,
       3, 3, 4, 3, 4, 4, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 4, 4, 4, 4,
       3, 3, 3, 3, 4, 3, 4, 3, 4, 4, 3, 3, 2, 4, 4, 2, 3, 3, 4, 4, 3, 4,
       3, 3, 4, 2, 4, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 4, 3, 2,
       4, 4, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 2, 2, 4, 3, 4, 2, 4, 3,
       3, 4, 3, 3, 3, 3, 4, 3, 4, 3, 4, 2, 2, 4, 3, 4, 3, 2, 4, 3, 3, 2,
       4, 4, 4, 4, 3, 0, 3, 3, 3, 3, 1, 4, 3, 3, 3, 4, 3, 4, 3, 3, 3, 4,
       3, 3, 4, 4, 4, 4, 3, 3, 4, 3, 4, 3, 4, 4, 3, 4, 4, 3, 4, 4, 3, 3,
     

データをシャッフルしてバッチ処理を行う。

In [9]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## モデルを作成して訓練する

In [10]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [11]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

Train for 303 steps
Epoch 1/15
303/303 [==============================] - 1s 4ms/step - loss: 0.9406 - accuracy: 0.7063
Epoch 2/15
303/303 [==============================] - 1s 2ms/step - loss: 0.6647 - accuracy: 0.7327
Epoch 3/15
303/303 [==============================] - 1s 2ms/step - loss: 0.6030 - accuracy: 0.7393
Epoch 4/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5611 - accuracy: 0.7459
Epoch 5/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5675 - accuracy: 0.7525
Epoch 6/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5533 - accuracy: 0.7591
Epoch 7/15
303/303 [==============================] - 1s 2ms/step - loss: 0.6182 - accuracy: 0.7228
Epoch 8/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5584 - accuracy: 0.7261
Epoch 9/15
303/303 [==============================] - 1s 2ms/step - loss: 0.5039 - accuracy: 0.7756
Epoch 10/15
303/303 [==============================] - 1s 2ms/step - loss: 0.560

## 特徴列の代替

モデルへの入力に辞書型データを渡すことは、`tf.keras.layers.Input`に同じ型の辞書を作成し、何らかの前処理を適用して`functional api`を使ってスタッキングすることと同様に、簡単に行うことができる。
これを特徴列(feature columns)の替わりに使うことができる。

In [13]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss='binary_crossentropy',
                   metrics=['accuracy'])

`tf.data`を使う時に、pandas の DataFrame の列構造を保持する一番簡単な方法は、DataFrame を辞書型データに変換して、先頭を切り取ることpandas の DataFrame の列構造を保持する一番簡単な方法は、DataFrame を辞書型データに変換して、先頭を切り取ること。

In [15]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)
for dict_slice in dict_slices.take(1):
  print (dict_slice)

({'age': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([63, 67, 67, 37, 41, 56, 62, 57, 63, 53, 57, 56, 56, 44, 52, 57])>, 'sex': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1])>, 'cp': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 4, 4, 3, 2, 2, 4, 4, 4, 4, 4, 2, 3, 2, 3, 3])>, 'trestbps': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([145, 160, 120, 130, 130, 120, 140, 120, 130, 140, 140, 140, 130,
       120, 172, 150])>, 'chol': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([233, 286, 229, 250, 204, 236, 268, 354, 254, 203, 192, 294, 256,
       263, 199, 168])>, 'fbs': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0])>, 'restecg': <tf.Tensor: shape=(16,), dtype=int32, numpy=array([2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0])>, 'thalach': <tf.Tensor: shape=(16,), dtype=int32, numpy=
array([150, 108, 129, 187, 172, 178, 160, 163, 147, 155, 148, 153, 142,
  

In [17]:
model_func.fit(dict_slices, epochs=15)

Train for 19 steps
Epoch 1/15
19/19 [==============================] - 0s 20ms/step - loss: 52.8702 - accuracy: 0.2739
Epoch 2/15
19/19 [==============================] - 0s 3ms/step - loss: 35.8708 - accuracy: 0.2739
Epoch 3/15
19/19 [==============================] - 0s 3ms/step - loss: 17.0307 - accuracy: 0.2739
Epoch 4/15
19/19 [==============================] - 0s 3ms/step - loss: 2.5448 - accuracy: 0.5974
Epoch 5/15
19/19 [==============================] - 0s 3ms/step - loss: 1.6373 - accuracy: 0.7459
Epoch 6/15
19/19 [==============================] - 0s 3ms/step - loss: 1.1861 - accuracy: 0.6832
Epoch 7/15
19/19 [==============================] - 0s 4ms/step - loss: 1.0891 - accuracy: 0.7030
Epoch 8/15
19/19 [==============================] - 0s 4ms/step - loss: 0.9973 - accuracy: 0.7063
Epoch 9/15
19/19 [==============================] - 0s 3ms/step - loss: 0.9375 - accuracy: 0.6997
Epoch 10/15
19/19 [==============================] - 0s 3ms/step - loss: 0.8633 - accuracy: 0.7